In [4]:
import pandas as pd
import numpy as np
import os
import requests
import base64
import csv
import pickle

In [5]:
train_dir = r'D:\AI\PART2\Cars Dataset\train'
test_dir = r'D:\AI\PART2\Cars Dataset\test'
Dataset = r'D:\AI\PART2\Cars Dataset'

In [6]:
data_set = os.listdir(Dataset)
print("รายชื่อไฟล์ในโฟลเดอร์ test:", data_set)

# แสดงรายชื่อไฟล์ในโฟลเดอร์ test
test_files = os.listdir(test_dir)
print("รายชื่อไฟล์ในโฟลเดอร์ test:", test_files)

# แสดงรายชื่อไฟล์ในโฟลเดอร์ train
train_files = os.listdir(train_dir)
print("รายชื่อไฟล์ในโฟลเดอร์ train:", train_files)


รายชื่อไฟล์ในโฟลเดอร์ test: ['test', 'train']
รายชื่อไฟล์ในโฟลเดอร์ test: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']
รายชื่อไฟล์ในโฟลเดอร์ train: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']


In [7]:
brands = {
        0:"Audi",
        1:"Hyundai Creta",
        2:"Mahindra Scorpio",
        3:'Rolls Royce',
        4:'Swift',
        5:'Tata Safari',
        6:'Toyota Innova'
        }

In [8]:
def getModelNumber(model_name):
    for number, name in brands.items():
        if name == model_name:
            return number
    return None 

In [9]:
def numeric_data(hog_data):        
    ss = hog_data.strip('[,]')  
    sss = ss.split(', ') 
    numeric_data = [float(value) for value in sss]
    return numeric_data

In [12]:
def GETHOG(part_dir):
    
    listData = [] # List to store training data 
    folder_path = part_dir  # Make sure to set test_dir to the appropriate path
    if not os.path.exists(folder_path):
        print("โฟลเดอร์ไม่มีอยู่")
        exit()
    Amount = 0
    for filebrand in os.listdir(folder_path):
        count = 0
      
        file_path = os.path.join(folder_path, filebrand)
        for image_filename in os.listdir(file_path):
            image_path = os.path.join(file_path, image_filename)

            with open(image_path, "rb") as image_file:
                image_data = image_file.read()
                base64_encoded = base64.b64encode(image_data).decode("utf-8")

                url = r'http://localhost:8080/api/gethog'
                data64 = {
                    "img": 'data:image/' + str(os.path.splitext(image_path)[1]) + ';base64,' + base64_encoded
                }

                headers = {"Content-Type": "application/json"}
                response = requests.get(url, json=data64, headers=headers)
                
                if response.status_code == 200:
                    count = count + 1
                    data = {
                        "image": os.path.basename(image_path),
                        "HOG":  response.json()["Hog"],
                        "Class": getModelNumber(filebrand)
                    }
                    
                    listData.append(data) 
                else:
                    print("Error:", response.status_code)
        print(str((filebrand))+'  '+filebrand+"  "+str(count))
        Amount = Amount + count
    print(f'Amount {Amount}')
    return listData

In [13]:
def loadCSV(namesFile, listDatas):
    csv_filename = namesFile + ".csv"
    with open(csv_filename, "w", newline="") as csv_file:
        fieldnames = ["image", "HOG", "Class"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in listDatas:
            # Convert the "HOG" data from a list of numbers to a string
            hog_data = ', '.join(str(number) for number in data["HOG"])
            # Write the data to the CSV file
            writer.writerow({"image": data["image"], "HOG": hog_data, "Class": data["Class"]})

In [14]:
def save_pkmax(namesFile, listDatas):
    pkmax_filename = namesFile + ".pkmax"
    with open(pkmax_filename, "wb") as pkmax_file:
        pickle.dump(listDatas, pkmax_file)

def load_pkmax(namesFile):
    pkmax_filename = namesFile + ".pkmax"
    with open(pkmax_filename, "rb") as pkmax_file:
        listDatas = pickle.load(pkmax_file)
    return listDatas

In [15]:
list_train = GETHOG(train_dir)

Audi  Audi  814
Hyundai Creta  Hyundai Creta  271
Mahindra Scorpio  Mahindra Scorpio  316
Rolls Royce  Rolls Royce  311
Swift  Swift  424
Tata Safari  Tata Safari  441
Toyota Innova  Toyota Innova  775
Amount 3352


In [16]:
list_test = GETHOG(test_dir)

Audi  Audi  199
Hyundai Creta  Hyundai Creta  67
Mahindra Scorpio  Mahindra Scorpio  75
Rolls Royce  Rolls Royce  74
Swift  Swift  102
Tata Safari  Tata Safari  106
Toyota Innova  Toyota Innova  190
Amount 813


In [17]:
loadCSV('train_data',list_train)
loadCSV('test_data',list_test)

In [28]:
save_pkmax("train_data", list_train)
save_pkmax("test_data", list_test)

In [63]:
data_train = load_pkmax("train_data")
data_test = load_pkmax("test_data")

In [29]:
data_path_train = f'D:\AI\PART2/train_data.csv'
data_path_test = f'D:\AI\PART2/test_data.csv'

importdata

In [30]:
data_train = pd.read_csv(data_path_train)
# data_train.head()
data_train.shape

(3352, 3)

In [31]:
data_test = pd.read_csv(data_path_test)
data_test.head()

,image,HOG,Class
0,1000.jpg,"0.011094658635556698, 0.012741324491798878, 0....",0
1,1003.jpg,"0.04191439598798752, 0.06864418834447861, 0.09...",0
2,1008.jpg,"0.08801346272230148, 0.021987106651067734, 0.0...",0
3,1020.jpg,"0.0021135981660336256, 0.05653822049498558, 0....",0
4,1022.jpg,"0.16168105602264404, 0.13524803519248962, 0.15...",0


***sklearn***

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree

In [33]:
X_train = data_train['HOG']
Y_train = data_train['Class']
print(Y_train.unique())

[0 1 2 3 4 5 6]


In [34]:
X_test = data_test['HOG']
Y_test = data_test['Class']
print(Y_test.unique())

[0 1 2 3 4 5 6]


In [47]:
def convert_hog_string_to_vector(hog_string):
    hog_list = eval(hog_string)
    hog_vector = np.array(hog_list)
    return hog_vector

In [52]:
X_train_numeric = np.array([convert_hog_string_to_vector(hog_string) for hog_string in X_train])
X_test_numeric = np.array([convert_hog_string_to_vector(hog_string) for hog_string in X_test])

In [53]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train_numeric,Y_train)

In [54]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import  accuracy_score   
y_model_prediction = clf.predict(X_test_numeric)

print(accuracy_score(Y_test,y_model_prediction)*100)
print(confusion_matrix(Y_test,y_model_prediction))

48.09348093480935
[[ 79  17  16  24  15  22  26]
 [  9  35   5   5   3   3   7]
 [  7   5  28   4   5   8  18]
 [ 28   7   7   9   7   8   8]
 [ 19   6   5   4  46   5  17]
 [  8   7   8   5   4  68   6]
 [ 19   6   9   8  10  12 126]]


In [55]:
path_model =f'D:\AI\PART2\image_modelv3.pk'
pickle.dump(clf, open(path_model,'wb'))